In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, to_date
from pyspark.sql.types import FloatType
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd
import pyspark.sql.functions as F
import matplotlib.pyplot as plt

In [6]:
import mysql.connector
import pandas as pd 
from pyspark.sql import SparkSession 
#appName = "PySpark MySQL Example - via mysql.connector" 
#master = "local" 
#spark=SparkSession.builder.master(master).appName(appName).getOrCreate() 
# Establish a connection 
conn=mysql.connector.connect(user='root', database='TrumpDB', password='password', host="localhost", port=3306) 
cursor = conn.cursor() 


In [ ]:
#query = "SELECT * FROM trumptable" 
query = "SELECT * FROM trumptable WHERE candidate_id = 2" 
# Create a pandas dataframe 
pdf = pd.read_sql(query, con=conn) 
conn.close() 
# Convert Pandas dataframe to spark DataFrame 
df = spark.createDataFrame(pdf) 
df.show()

/tmp/ipykernel_8657/4065591516.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pdf = pd.read_sql(query, con=conn)


In [2]:
jdbcDF = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://localhost:3306/TrumpDB") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", "trumptable") \
    .option("user", "root") \
    .option("password", "password") \
    .load()

jdbcDF.show()



ERROR:root:Exception while sending command.                         (0 + 1) / 1]
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


Py4JError: An error occurred while calling o42.showString

In [ ]:
filename = '2016_US_election_tweets.csv'

# Read input file from hadoop directory and convert to pandas
df = spark.read.csv('/user1/CA2/'+filename,header=True)


In [ ]:
print(df.columns)

In [ ]:
df = df.select('candidate_id', "created_at", "tweet_text")

In [ ]:
df_no_nulls = df.na.drop()
tweets = df_no_nulls.filter(df.candidate_id == 2)

In [ ]:
analyzer = SentimentIntensityAnalyzer()

def analyze_sentiment(text):
    return analyzer.polarity_scores(text)["compound"]

sentiment_udf = udf(analyze_sentiment, FloatType())
spark.udf.register("sentiment_udf", sentiment_udf)

In [ ]:
# Convert timestamp to date
tweets = tweets.withColumn("date", to_date(col("created_at")))

# Calculate sentiment score for each tweet
tweets = tweets.withColumn("sentiment", sentiment_udf(col("tweet_text")))


In [ ]:
tweets = tweets.select('tweet_text','date','sentiment')

tweets.show()

In [ ]:
from pyspark.sql.functions import year

# filter out out-of-range dates
tweets = tweets.filter((year("date") >= 1) & (year("date") <= 9999))

In [ ]:
daily_sentiment_df = tweets.groupBy("date") \
    .agg(F.mean("sentiment").alias("avg_sentiment")) \
    .orderBy("date")

In [ ]:
daily_sentiment_pd = daily_sentiment_df.toPandas()

In [ ]:
df2 = daily_sentiment_pd.iloc[:194,:]

In [ ]:
df2['date'] = pd.to_datetime(df2['date'])

In [ ]:
print(df2['date'])

In [ ]:
df2 = df2[(df2['date'].dt.year >= 2016) & (df2['date'].dt.year <= 2017)]

In [ ]:
print(df2)

In [ ]:
df2 = df2.iloc[1:]

In [ ]:
plt.plot(df2['date'],df2['avg_sentiment'])

In [ ]:
import plotly.express as px

In [ ]:
fig = px.line(x = df2.date,y = df2.avg_sentiment)
fig.show()


In [ ]:
[print(x) for x in df2['date']] 